In [1]:
import pandas as pd

#read the data
rock_samples=pd.read_csv('data/rocksamples.csv')

In [2]:
#print top 5 rows of data
rock_samples.head()

,ID,Mission,Type,Subtype,Weight(g),Pristine(%)
0,10001,Apollo11,Soil,Unsieved,125.8,88.36
1,10002,Apollo11,Soil,Unsieved,5629.0,93.73
2,10003,Apollo11,Basalt,Ilmenite,213.0,65.56
3,10004,Apollo11,Core,Unsieved,44.8,71.76
4,10005,Apollo11,Core,Unsieved,53.4,40.31


In [3]:
#get the information summary
rock_samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2229 entries, 0 to 2228
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           2229 non-null   int64  
 1   Mission      2229 non-null   object 
 2   Type         2229 non-null   object 
 3   Subtype      2226 non-null   object 
 4   Weight(g)    2229 non-null   float64
 5   Pristine(%)  2229 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 104.6+ KB


There are 2229 samples collected:   
- ID - The unique ID used to keep track of the sample at NASA.
- Mission - The mission responsible for retrieving the sample.
- Type - The type of sample (type of rock or other classification).
-  Subtype - A more specific type classification.
- Weight(g) - The original weight of the sample, in grams.
- Pristine(%) - The percentage of the sample that remains (some sample is used up during research).


## Data Cleaning

### Convert the sample weight

Rocket Weight is often measures in kilograms, not grams. Converting the sample weights to kilograms will make data analysis easier

In [4]:
#converting from grams to kg
rock_samples['Weight(g)']=rock_samples['Weight(g)'].apply(lambda x:x *0.001)
#rename the column
rock_samples.rename(columns={'Weight(g)':'Weight(kg)'}, inplace=True)
rock_samples.head()

,ID,Mission,Type,Subtype,Weight(kg),Pristine(%)
0,10001,Apollo11,Soil,Unsieved,0.1258,88.36
1,10002,Apollo11,Soil,Unsieved,5.6290,93.73
2,10003,Apollo11,Basalt,Ilmenite,0.2130,65.56
3,10004,Apollo11,Core,Unsieved,0.0448,71.76
4,10005,Apollo11,Core,Unsieved,0.0534,40.31


### Create a new Dataframe

The rock_samples dataframe has a row for every sample that was collected but we want to understand the rock samples in total as they relate to the specific rockets that brought them back.
We create a new dataframe that will be summary data for each of the six Apollo missions that brought samples back

In [5]:
missions=pd.DataFrame()
missions['Mission']=rock_samples['Mission'].unique()
missions.head()

,Mission
0,Apollo11
1,Apollo12
2,Apollo14
3,Apollo15
4,Apollo16


In [6]:
#get the information summary
missions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Mission  6 non-null      object
dtypes: object(1)
memory usage: 176.0+ bytes


### Sum total sample weight by mission

Lets add the total number of samples collected by each mission to the dataframe

In [7]:
#get the sum of each mission
sample_total_weight=rock_samples.groupby('Mission')['Weight(kg)'].sum()
#add the information to the missions dataframe
missions=pd.merge(missions,sample_total_weight,on='Mission')
missions.rename(columns={'Weight(kg)':'Sample Weight(kg)'},inplace=True)
missions

,Mission,Sample Weight(kg)
0,Apollo11,21.55424
1,Apollo12,34.34238
2,Apollo14,41.83363
3,Apollo15,75.39910
4,Apollo16,92.46262
5,Apollo17,109.44402


### Get the difference in weights across missions


From the information above, we can see that there was an increase in weight of samples with each mission. However we can't tell by how much. Let's get the difference and add a column

In [8]:
#getting the difference between column values
missions['Weight Diff']=missions['Sample Weight(kg)'].diff()
missions

,Mission,Sample Weight(kg),Weight Diff
0,Apollo11,21.55424,NaN
1,Apollo12,34.34238,12.78814
2,Apollo14,41.83363,7.49125
3,Apollo15,75.39910,33.56547
4,Apollo16,92.46262,17.06352
5,Apollo17,109.44402,16.98140


Because Apollo11 was the 1st mission, the value in the Weight Diff column is NAN. We can replace it with a zero.

In [9]:
#replacing the Nan value with 0
missions['Weight Diff']=missions['Weight Diff'].fillna(value=0)
missions

,Mission,Sample Weight(kg),Weight Diff
0,Apollo11,21.55424,0.00000
1,Apollo12,34.34238,12.78814
2,Apollo14,41.83363,7.49125
3,Apollo15,75.39910,33.56547
4,Apollo16,92.46262,17.06352
5,Apollo17,109.44402,16.98140


### Add in command and lunar module data


There are two modules that ensure astronauts safely enter the moon's orbit and return to earth safely. (Lunar Module, Command Module)
By using the NASA Space Science Data Coordinated Archive , we gathered information about each module used in each mission. Let's add them to our missions dataframe
- Module Name
- Module Mass
- Module Mass Diff

In [10]:
missions['Lunar Module (LM)'] = {'Eagle (LM-5)', 'Intrepid (LM-6)', 'Antares (LM-8)', 'Falcon (LM-10)', 'Orion (LM-11)', 'Challenger (LM-12)'}
missions['LM Mass (kg)'] = {15103, 15235, 15264, 16430, 16445, 16456}
missions['LM Mass Diff'] = missions['LM Mass (kg)'].diff()
missions['LM Mass Diff'] = missions['LM Mass Diff'].fillna(value=0)

missions['Command Module (CM)'] = {'Columbia (CSM-107)', 'Yankee Clipper (CM-108)', 'Kitty Hawk (CM-110)', 'Endeavor (CM-112)', 'Casper (CM-113)', 'America (CM-114)'}
missions['CM Mass (kg)'] = {5560, 5609, 5758, 5875, 5840, 5960}
missions['CM Mass Diff'] = missions['CM Mass (kg)'].diff()
missions['CM Mass Diff'] = missions['CM Mass Diff'].fillna(value=0)
missions


,Mission,Sample Weight(kg),Weight Diff,Lunar Module (LM),LM Mass (kg),LM Mass Diff,Command Module (CM),CM Mass (kg),CM Mass Diff
0,Apollo11,21.55424,0.00000,Intrepid (LM-6),15264,0.0,America (CM-114),5960,0.0
1,Apollo12,34.34238,12.78814,Eagle (LM-5),15235,-29.0,Yankee Clipper (CM-108),5609,-351.0
2,Apollo14,41.83363,7.49125,Orion (LM-11),16456,1221.0,Columbia (CSM-107),5840,231.0
3,Apollo15,75.39910,33.56547,Falcon (LM-10),16430,-26.0,Casper (CM-113),5875,35.0
4,Apollo16,92.46262,17.06352,Challenger (LM-12),16445,15.0,Kitty Hawk (CM-110),5560,-315.0
5,Apollo17,109.44402,16.98140,Antares (LM-8),15103,-1342.0,Endeavor (CM-112),5758,198.0


Let's get the total for some missions across both the lunar and command modules


In [11]:
missions['Total Weight (kg)'] = missions['LM Mass (kg)'] + missions['CM Mass (kg)']
missions['Total Weight Diff'] = missions['LM Mass Diff'] + missions['CM Mass Diff']
missions

,Mission,Sample Weight(kg),Weight Diff,Lunar Module (LM),LM Mass (kg),LM Mass Diff,Command Module (CM),CM Mass (kg),CM Mass Diff,Total Weight (kg),Total Weight Diff
0,Apollo11,21.55424,0.00000,Intrepid (LM-6),15264,0.0,America (CM-114),5960,0.0,21224,0.0
1,Apollo12,34.34238,12.78814,Eagle (LM-5),15235,-29.0,Yankee Clipper (CM-108),5609,-351.0,20844,-380.0
2,Apollo14,41.83363,7.49125,Orion (LM-11),16456,1221.0,Columbia (CSM-107),5840,231.0,22296,1452.0
3,Apollo15,75.39910,33.56547,Falcon (LM-10),16430,-26.0,Casper (CM-113),5875,35.0,22305,9.0
4,Apollo16,92.46262,17.06352,Challenger (LM-12),16445,15.0,Kitty Hawk (CM-110),5560,-315.0,22005,-300.0
5,Apollo17,109.44402,16.98140,Antares (LM-8),15103,-1342.0,Endeavor (CM-112),5758,198.0,20861,-1144.0


### Compare the data

A payload is basically the total amount of weight that a rocket can get up through our atmosphere and into space. So the likelihood that the payload number is more accurate than the exact weights of each module is high, because deciding the payload will likely affect each of the other design decisions.

We know that the Saturn V payload was 43,500 kg, and the weights of the modules varied from mission to mission. So, to determine the ratios that will allow us to make predictions about the Artemis missions, we can use:
- Saturn V payload
- Mission sample weight
- Mission module weight

In [12]:
#sample to weight ratio
saturnVPayload=43500
missions['Crewed Area: payload']=missions['Total Weight (kg)']/saturnVPayload
missions['Sample: Crewed Area']=missions['Sample Weight(kg)']/missions['Total Weight (kg)']
missions['Sample: Payload']=missions['Sample Weight(kg)']/ saturnVPayload
missions

,Mission,Sample Weight(kg),Weight Diff,Lunar Module (LM),LM Mass (kg),LM Mass Diff,Command Module (CM),CM Mass (kg),CM Mass Diff,Total Weight (kg),Total Weight Diff,Crewed Area: payload,Sample: Crewed Area,Sample: Payload
0,Apollo11,21.55424,0.00000,Intrepid (LM-6),15264,0.0,America (CM-114),5960,0.0,21224,0.0,0.487908,0.001016,0.000495
1,Apollo12,34.34238,12.78814,Eagle (LM-5),15235,-29.0,Yankee Clipper (CM-108),5609,-351.0,20844,-380.0,0.479172,0.001648,0.000789
2,Apollo14,41.83363,7.49125,Orion (LM-11),16456,1221.0,Columbia (CSM-107),5840,231.0,22296,1452.0,0.512552,0.001876,0.000962
3,Apollo15,75.39910,33.56547,Falcon (LM-10),16430,-26.0,Casper (CM-113),5875,35.0,22305,9.0,0.512759,0.003380,0.001733
4,Apollo16,92.46262,17.06352,Challenger (LM-12),16445,15.0,Kitty Hawk (CM-110),5560,-315.0,22005,-300.0,0.505862,0.004202,0.002126
5,Apollo17,109.44402,16.98140,Antares (LM-8),15103,-1342.0,Endeavor (CM-112),5758,198.0,20861,-1144.0,0.479563,0.005246,0.002516


### Save the ratios
We can then use the mean() function to take the average of all those ratios across all the missions

In [13]:
crewedArea_payload_ratio = missions['Crewed Area: payload'].mean()
sample_crewedArea_ratio = missions['Sample: Crewed Area'].mean()
sample_payload_ratio = missions['Sample: Payload'].mean()
print(crewedArea_payload_ratio)
print(sample_crewedArea_ratio)
print(sample_payload_ratio)

0.49630268199233724
0.0028946732226251396
0.0014369195019157093


These ratios can be used to predict the Artemis capacity for the samples

## Predict Artemis Sample Capacity
The Artemis program  is NASA's second set of missions to land humans on the surface of the Moon. The program will launch in 2024. There will be three iterations of the rocket cycled through each mission.

### Creating an Artemis mission dataframe
We will call the command and lunar modules the crewed area, and we can create a dataframe with the information we have about the three crewed missions

In [14]:
artemis_crewedArea = 26520
artemis_mission = pd.DataFrame({'Mission':['artemis1','artemis1b','artemis2'],                                 'Total Weight (kg)':[artemis_crewedArea,artemis_crewedArea,artemis_crewedArea],                                 'Payload (kg)':[26988, 37965, 42955]})
artemis_mission

,Mission,Total Weight (kg),Payload (kg)
0,artemis1,26520,26988
1,artemis1b,26520,37965
2,artemis2,26520,42955


We can then estimate the weight of samples based on the ratios we determined from the Artemis missions

In [15]:
artemis_mission['Sample Weight from Total (kg)'] = artemis_mission['Total Weight (kg)'] * sample_crewedArea_ratio
artemis_mission['Sample Weight from Payload (kg)'] = artemis_mission['Payload (kg)'] * sample_payload_ratio
artemis_mission

,Mission,Total Weight (kg),Payload (kg),Sample Weight from Total (kg),Sample Weight from Payload (kg)
0,artemis1,26520,26988,76.766734,38.779584
1,artemis1b,26520,37965,76.766734,54.552649
2,artemis2,26520,42955,76.766734,61.722877


Now we can get the average of the two predictions

In [16]:
artemis_mission['Estimated Sample Weight (kg)'] = (artemis_mission['Sample Weight from Payload (kg)'] + artemis_mission['Sample Weight from Total (kg)'])/2
artemis_mission

,Mission,Total Weight (kg),Payload (kg),Sample Weight from Total (kg),Sample Weight from Payload (kg),Estimated Sample Weight (kg)
0,artemis1,26520,26988,76.766734,38.779584,57.773159
1,artemis1b,26520,37965,76.766734,54.552649,65.659691
2,artemis2,26520,42955,76.766734,61.722877,69.244806


We can see now that the three Artemis missions can likely return 57.77 kg, 65.65 kg, and 69.24 kg, respectively.
Next we check for the kinds of rocks that they should prioritize.

### Prioritize Moon rock sample gathering based on data
First, we can determine how much remains of each sample that was returned from the Apollo missions, given the amount that was originally collected and the percentage of remaining pristine sample.

In [17]:
rock_samples['Remaining(kg)'] = rock_samples['Weight(kg)'] * (rock_samples['Pristine(%)'] * .01)
rock_samples.head()

,ID,Mission,Type,Subtype,Weight(kg),Pristine(%),Remaining(kg)
0,10001,Apollo11,Soil,Unsieved,0.1258,88.36,0.111157
1,10002,Apollo11,Soil,Unsieved,5.6290,93.73,5.276062
2,10003,Apollo11,Basalt,Ilmenite,0.2130,65.56,0.139643
3,10004,Apollo11,Core,Unsieved,0.0448,71.76,0.032148
4,10005,Apollo11,Core,Unsieved,0.0534,40.31,0.021526


In [18]:
rock_samples.describe()

,ID,Weight(kg),Pristine(%),Remaining(kg)
count,2229.000000,2229.000000,2229.000000,2229.000000
mean,52058.432032,0.168253,84.512764,0.138103
std,26207.651471,0.637286,22.057299,0.525954
min,10001.000000,0.000000,0.000000,0.000000
25%,15437.000000,0.003000,80.010000,0.002432
50%,65527.000000,0.010200,92.300000,0.008530
75%,72142.000000,0.093490,98.140000,0.078240
max,79537.000000,11.729000,180.000000,11.169527


This helps us see that, on average, each sample weighs about .16 kg and has about 84% of the original amount remaining. We can use this knowledge to extract only the samples that are likely running low, which means that they have been used a lot by researchers.



In [19]:
low_samples = rock_samples.loc[(rock_samples['Weight(kg)'] >= .16) & (rock_samples['Pristine(%)'] <= 50)]
low_samples.head()

,ID,Mission,Type,Subtype,Weight(kg),Pristine(%),Remaining(kg)
11,10017,Apollo11,Basalt,Ilmenite,0.973,43.71,0.425298
14,10020,Apollo11,Basalt,Ilmenite,0.425,27.88,0.118490
15,10021,Apollo11,Breccia,Regolith,0.250,30.21,0.075525
29,10045,Apollo11,Basalt,Olivine,0.185,12.13,0.022441
37,10057,Apollo11,Basalt,Ilmenite,0.919,35.15,0.323028


In [20]:
low_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 11 to 2183
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             27 non-null     int64  
 1   Mission        27 non-null     object 
 2   Type           27 non-null     object 
 3   Subtype        27 non-null     object 
 4   Weight(kg)     27 non-null     float64
 5   Pristine(%)    27 non-null     float64
 6   Remaining(kg)  27 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.7+ KB


we can use the unique() function to see how many unique types we have across the low_samples and rock_samples dataframes.

In [21]:
low_samples.Type.unique()

array(['Basalt', 'Breccia', 'Soil', 'Core'], dtype=object)

In [22]:
rock_samples.Type.unique()

array(['Soil', 'Basalt', 'Core', 'Breccia', 'Special', 'Crustal'],
      dtype=object)

The samples that are running low are from only four unique types. But this doesn't tell us everything about the samples we might want to focus on. For example, in our low_samples dataframe, how many of each type are actually considered low?

In [23]:
low_samples.groupby('Type')['Weight(kg)'].count()

Type
Basalt     14
Breccia     8
Core        1
Soil        4
Name: Weight(kg), dtype: int64

In [24]:
needed_samples = low_samples[low_samples['Type'].isin(['Basalt', 'Breccia'])]
needed_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 11 to 2183
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             22 non-null     int64  
 1   Mission        22 non-null     object 
 2   Type           22 non-null     object 
 3   Subtype        22 non-null     object 
 4   Weight(kg)     22 non-null     float64
 5   Pristine(%)    22 non-null     float64
 6   Remaining(kg)  22 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.4+ KB


#### Develop a recommendation of Moon rock samples
Let's take a step back and see how the number of samples compares to the amount of sample. We can compare the total weight from the needed_samples dataframe to the rock_samples dataframe. That is, we'll compare the samples we've identified as running low to all the samples collected on Apollo missions.

In [25]:
needed_samples.groupby('Type')['Weight(kg)'].sum()

Type
Basalt     17.4234
Breccia    10.1185
Name: Weight(kg), dtype: float64

In [26]:
rock_samples.groupby('Type')['Weight(kg)'].sum()

Type
Basalt      93.14077
Breccia    168.88075
Core        19.93587
Crustal      4.74469
Soil        87.58981
Special      0.74410
Name: Weight(kg), dtype: float64

One bit of information really stands out: we've never had a lot of Crustal rocks in the first place.

We can add Crustal rocks to the set of needed samples:

In [27]:
needed_samples = needed_samples.append(rock_samples.loc[rock_samples['Type'] == 'Crustal'])
needed_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 11 to 2189
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             68 non-null     int64  
 1   Mission        68 non-null     object 
 2   Type           68 non-null     object 
 3   Subtype        68 non-null     object 
 4   Weight(kg)     68 non-null     float64
 5   Pristine(%)    68 non-null     float64
 6   Remaining(kg)  68 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 4.2+ KB


#### Summary of needed samples

In [28]:
needed_samples_overview = pd.DataFrame()
needed_samples_overview['Type'] = needed_samples.Type.unique()
needed_samples_overview

,Type
0,Basalt
1,Breccia
2,Crustal


Next, we want the total weight of each type of rock that was originally collected:



In [29]:
needed_sample_weights = needed_samples.groupby('Type')['Weight(kg)'].sum().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, needed_sample_weights, on='Type')
needed_samples_overview.rename(columns={'Weight(kg)':'Total Weight(kg)'}, inplace=True)
needed_samples_overview

,Type,Total Weight(kg)
0,Basalt,17.42340
1,Breccia,10.11850
2,Crustal,4.74469


In [30]:
needed_sample_ave_weights = needed_samples.groupby('Type')['Weight(kg)'].mean().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, needed_sample_ave_weights, on='Type')
needed_samples_overview.rename(columns={'Weight(kg)':'Ave Weight(kg)'}, inplace=True)
needed_samples_overview

,Type,Total Weight(kg),Ave Weight(kg)
0,Basalt,17.42340,1.244529
1,Breccia,10.11850,1.264812
2,Crustal,4.74469,0.103145


In [31]:
total_rock_count = rock_samples.groupby('Type')['ID'].count().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, total_rock_count, on='Type')
needed_samples_overview.rename(columns={'ID':'Number of Samples'}, inplace=True)
total_rocks = needed_samples_overview['Number of Samples'].sum()
needed_samples_overview['Percentage of Rocks'] = needed_samples_overview['Number of Samples'] / total_rocks
needed_samples_overview

,Type,Total Weight(kg),Ave Weight(kg),Number of Samples,Percentage of Rocks
0,Basalt,17.42340,1.244529,351,0.258850
1,Breccia,10.11850,1.264812,959,0.707227
2,Crustal,4.74469,0.103145,46,0.033923


And finally, to tie it all back into a recommendation to the Artemis program, we can determine the average weight of samples we estimated in the preceding unit.

In [32]:
artemis_ave_weight = artemis_mission['Estimated Sample Weight (kg)'].mean()
artemis_ave_weight

64.22588520079607

We can use this number to determine how many of each rock we want the astronauts to aim to collect:



In [33]:
needed_samples_overview['Weight to Collect'] = needed_samples_overview['Percentage of Rocks'] * artemis_ave_weight

needed_samples_overview['Rocks to Collect'] = needed_samples_overview['Weight to Collect'] / needed_samples_overview['Ave Weight(kg)']

needed_samples_overview

,Type,Total Weight(kg),Ave Weight(kg),Number of Samples,Percentage of Rocks,Weight to Collect,Rocks to Collect
0,Basalt,17.42340,1.244529,351,0.258850,16.624842,13.358345
1,Breccia,10.11850,1.264812,959,0.707227,45.422289,35.912271
2,Crustal,4.74469,0.103145,46,0.033923,2.178754,21.123128


So, we might tell the Artemis astronauts to please try to collect 13 Basalt rocks, 35 Breccia rocks, and 21 Crustal rocks. Whew!

